In [ ]:
!pip install transformers seqeval[gpu]

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 6.5 MB 30.2 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 895 kB 50.4 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=adcb74a75ef5a6c502919b0648260124b66f504ffa090d8a0ce59cc1a301e191
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [22]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [23]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
data = pd.read_csv("/content/drive/MyDrive/BIO_tagged_dataset_Modified2.csv")

In [26]:
df = pd.DataFrame(columns = ['Sentence #', 'Word', 'Tag'])
i = 0
sentence_number = 1
for index, row in data.iterrows():
  tokens = row["text"].split(" ")
  tags = row["BIO_tag"].split(" ")
  for j in range(len(tokens)):
    df.loc[i] = ["Sentence: {}".format(sentence_number)] + [tokens[j]] + [tags[j]]
    print(df.loc[i])
    j = j + 1
    i = i + 1
  sentence_number = sentence_number + 1

Streaming output truncated to the last 5000 lines.
Sentence #    Sentence: 487
Word                 smooth
Tag                       O
Name: 77945, dtype: object
Sentence #    Sentence: 487
Word              solutions
Tag                       O
Name: 77946, dtype: object
Sentence #    Sentence: 487
Word             considered
Tag                       O
Name: 77947, dtype: object
Sentence #    Sentence: 487
Word                     as
Tag                       O
Name: 77948, dtype: object
Sentence #    Sentence: 487
Word                   test
Tag                       O
Name: 77949, dtype: object
Sentence #    Sentence: 487
Word                  cases
Tag                       O
Name: 77950, dtype: object
Sentence #    Sentence: 487
Word                    the
Tag                       O
Name: 77951, dtype: object
Sentence #    Sentence: 487
Word                results
Tag                       O
Name: 77952, dtype: object
Sentence #    Sentence: 487
Word                   show
Tag  

In [27]:
df.head()

,Sentence #,Word,Tag
0,Sentence: 1,several,O
1,Sentence: 1,inorganic,B
2,Sentence: 1,flocculating,I
3,Sentence: 1,agents,I
4,Sentence: 1,including,O


In [30]:
entities_to_remove = ["", " "]
df = df[~df.Tag.isin(entities_to_remove)]
df.head()

,Sentence #,Word,Tag
0,Sentence: 1,several,O
1,Sentence: 1,inorganic,B
2,Sentence: 1,flocculating,I
3,Sentence: 1,agents,I
4,Sentence: 1,including,O


In [31]:
labels_to_ids = {k: v for v, k in enumerate(df.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(df.Tag.unique())}
labels_to_ids

{'B': 1, 'I': 2, 'O': 0}

In [32]:
# let's create a new column called "sentence" which groups the words by sentence 
df['sentence'] = df[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence 
df['word_labels'] = df[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))

In [33]:
df.head()

,Sentence #,Word,Tag,sentence,word_labels
0,Sentence: 1,several,O,several inorganic flocculating agents includin...,"O,B,I,I,O,B,B,O,O,B,I,I,B,O,O,O,B,I,I,O,O,O,O,..."
1,Sentence: 1,inorganic,B,several inorganic flocculating agents includin...,"O,B,I,I,O,B,B,O,O,B,I,I,B,O,O,O,B,I,I,O,O,O,O,..."
2,Sentence: 1,flocculating,I,several inorganic flocculating agents includin...,"O,B,I,I,O,B,B,O,O,B,I,I,B,O,O,O,B,I,I,O,O,O,O,..."
3,Sentence: 1,agents,I,several inorganic flocculating agents includin...,"O,B,I,I,O,B,B,O,O,B,I,I,B,O,O,O,B,I,I,O,O,O,O,..."
4,Sentence: 1,including,O,several inorganic flocculating agents includin...,"O,B,I,I,O,B,B,O,O,B,I,I,B,O,O,O,B,I,I,O,O,O,O,..."


In [34]:
df = df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
df.head()

,sentence,word_labels
0,several inorganic flocculating agents includin...,"O,B,I,I,O,B,B,O,O,B,I,I,B,O,O,O,B,I,I,O,O,O,O,..."
1,video oculography is one of eye movement measu...,"B,I,O,O,O,B,I,I,O,O,O,O,B,O,O,B,I,I,I,I,O,B,I,..."
2,the first principles calculations are performe...,"O,B,I,I,O,O,O,O,B,I,I,I,I,O,O,O,B,I,O,O,O,O,B,..."
3,microhardness can be related to other macrosco...,"B,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B,I,O,B,..."
4,we consider finite time future or big rip sing...,"O,O,B,I,O,O,O,O,O,O,O,O,O,O,B,I,I,O,O,O,O,O,O,..."


In [35]:
len(df)

493

In [93]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [94]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.sentence[index].strip().split()  
        word_labels = self.data.word_labels[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [95]:
train_size = 0.8
train_dataset = df.sample(frac=train_size,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (493, 2)
TRAIN Dataset: (394, 2)
TEST Dataset: (99, 2)


In [96]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [97]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [98]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)


In [99]:
outputs= model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss.item()

1.201375961303711

In [100]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 3])

In [101]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [102]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = out[0]
        tr_logits = out[1]
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [103]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.1361240148544312
Training loss epoch: 0.5434401772841059
Training accuracy epoch: 0.7896792950102467
Training epoch: 2
Training loss per 100 training steps: 0.368062287569046
Training loss epoch: 0.4164448615276452
Training accuracy epoch: 0.8238953471449553
Training epoch: 3
Training loss per 100 training steps: 0.3975338935852051
Training loss epoch: 0.3722711294287383
Training accuracy epoch: 0.8432107212449139
Training epoch: 4
Training loss per 100 training steps: 0.37944939732551575
Training loss epoch: 0.34035953412754366
Training accuracy epoch: 0.8575670965434233
Training epoch: 5
Training loss per 100 training steps: 0.2885509431362152
Training loss epoch: 0.3041312322773115
Training accuracy epoch: 0.873861615319061
Training epoch: 6
Training loss per 100 training steps: 0.2224540412425995
Training loss epoch: 0.268739997437506
Training accuracy epoch: 0.8909320305708733
Training epoch: 7
Training loss per 100 trainin

In [104]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            out = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = out[0]
            eval_logits = out[1]
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [105]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.5581438541412354
Validation Loss: 0.6227742841839791
Validation Accuracy: 0.817811730820422


In [117]:
from sklearn.metrics  import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           B       0.50      0.51      0.51      1156
           I       0.51      0.52      0.51      1995
           O       0.90      0.89      0.90     12526

    accuracy                           0.82     15677
   macro avg       0.64      0.64      0.64     15677
weighted avg       0.82      0.82      0.82     15677

